In [1]:
#import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
#import scipy.stats as stats
#import statsmodels.api as sm
#pd.options.display.max_rows = None
#pd.options.display.max_columns = None
from datetime import timedelta
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
#Read in the data filtered down to active = true
parse_dates = ['EventTimeStamp','LocationTimeStamp']
faults = pd.read_csv('../data/filtered_df.csv', parse_dates = parse_dates)

C:\Users\maggi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
faults[faults['spn'].isin([4339])].count()

RecordID                 696
ESS_Id                   696
EventTimeStamp           696
eventDescription           0
ecuSoftwareVersion       693
ecuSerialNumber          668
ecuModel                 668
ecuMake                  668
ecuSource                696
spn                      696
fmi                      696
active                   696
activeTransitionCount    696
EquipmentID              696
MCTNumber                696
Latitude                 696
Longitude                696
LocationTimeStamp        696
dtype: int64

In [4]:
faults['EquipmentID'] = faults['EquipmentID'].astype('str')

In [5]:
faults['ecuMake'] = faults['ecuMake'].fillna('')
faults = faults[faults['ecuMake'].str.endswith('MNS')]

In [6]:
faults.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208380 entries, 3 to 577229
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   RecordID               208380 non-null  int64         
 1   ESS_Id                 208380 non-null  int64         
 2   EventTimeStamp         208380 non-null  datetime64[ns]
 3   eventDescription       200960 non-null  object        
 4   ecuSoftwareVersion     208200 non-null  object        
 5   ecuSerialNumber        208380 non-null  object        
 6   ecuModel               208380 non-null  object        
 7   ecuMake                208380 non-null  object        
 8   ecuSource              208380 non-null  int64         
 9   spn                    208380 non-null  int64         
 10  fmi                    208380 non-null  int64         
 11  active                 208380 non-null  bool          
 12  activeTransitionCount  208380 non-null  int6

In [7]:
#set a time window for how far in advance we want to predict a derate
failure_window = 2
#set a time window for aggregating the count of faults per EquipmentID in days
summary_window = 2
#print(max_date_calc)
#backfill the entries prior to each fault
faults['NextDerateTimestamp'] = faults[faults['spn']==5246]['EventTimeStamp']
faults = faults.sort_values(['EquipmentID','EventTimeStamp']).reset_index(drop=True)
faults['NextDerateTimestamp'] = \
    faults.sort_values(['EquipmentID','EventTimeStamp']).groupby('EquipmentID')['NextDerateTimestamp'].bfill()
#calculate the last time stamp for each EquipmentID and merge
MaxTimestamp = pd.DataFrame(faults.groupby('EquipmentID')['EventTimeStamp'].max())
MaxTimestamp = MaxTimestamp.rename(columns={'EventTimeStamp':'MaxTimestamp'})
faults = faults.merge(MaxTimestamp, how='left', on='EquipmentID')
#faults['NextDerateTimestamp'] = faults['NextDerateTimestamp'].fillna(faults['MaxTimestamp'])
#Calculate the date diff with NextDerateTimestamp (or, if null with MaxTimestamp)
faults['DateDiff'] = np.where(faults['NextDerateTimestamp'].isna(), 
                              (faults['MaxTimestamp'] - faults['EventTimeStamp']).dt.days ,
                              (faults['NextDerateTimestamp'] - faults['EventTimeStamp']).dt.days)
#If NextDerateTimestamp isna, discard recs within failure window--we have no way to know what happens after MaxTimestamp
#so these observations are not valid for prediction
faults = faults[(faults['NextDerateTimestamp'].notnull()) | (faults['DateDiff'] > failure_window)]
#Mark records as in or out of the failure window
faults['InFailureWindow'] = np.where(faults['DateDiff'] <= failure_window, 1, 0)
#forwardfill the entries after each fault
faults['PrevDerateTimestamp'] = faults[faults['spn']==5246]['EventTimeStamp']
faults['PrevDerateTimestamp'] = \
    faults.sort_values(['EquipmentID','EventTimeStamp']).groupby('EquipmentID')['PrevDerateTimestamp'].ffill()
#calculate the first time stamp for each EquipmentID and merge
MinTimestamp = pd.DataFrame(faults.groupby('EquipmentID')['EventTimeStamp'].min())
MinTimestamp = MinTimestamp.rename(columns={'EventTimeStamp':'MinTimestamp'})
faults = faults.merge(MinTimestamp, how='left', on='EquipmentID')
#Calculate the date diff with PrevDerateTimestamp (or, if null with MinTimestamp)
faults['DateDiffMin'] = np.where(faults['PrevDerateTimestamp'].isna(), 
                              (faults['EventTimeStamp'] - faults['MinTimestamp']).dt.days ,
                              (faults['EventTimeStamp'] - faults['PrevDerateTimestamp']).dt.days)
#Mark records as in or out of the summary window
#Has it been at least summary-window days since the last derate? 
#If so we can apply a summary-window-day summary to this EventTimeStamp
faults['InSummWindow'] = np.where(faults['DateDiffMin'] >= summary_window, 1, 0)
faults.head()


,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,...,Longitude,LocationTimeStamp,NextDerateTimestamp,MaxTimestamp,DateDiff,InFailureWindow,PrevDerateTimestamp,MinTimestamp,DateDiffMin,InSummWindow
0,4968,1059892,2015-02-25 06:17:50,Low (Severity Low) Engine Coolant Level,04993120*00001782*082113134117*07700053*I0*BBZ*,79419774,6X1u10D1500000000,CMMNS,0,111,...,-86.433981,2015-02-25 06:22:30,NaT,2015-06-08 19:37:07,103.0,0,NaT,2015-02-25 06:17:50,0.0,0
1,4969,1059893,2015-02-25 06:17:50,Low (Severity Medium) Battery Potential / Powe...,04993120*00001782*082113134117*07700053*I0*BBZ*,79419774,6X1u10D1500000000,CMMNS,0,444,...,-86.433981,2015-02-25 06:22:31,NaT,2015-06-08 19:37:07,103.0,0,NaT,2015-02-25 06:17:50,0.0,0
2,58809,2507532,2015-05-19 12:02:55,Abnormal Update Rate Aftertreatment 1 Intake NOx,04993120*00001782*082113134117*07700053*I0*BBZ*,79419774,6X1u10D1500000000,CMMNS,0,3216,...,-86.458842,2015-05-19 12:03:32,NaT,2015-06-08 19:37:07,20.0,0,NaT,2015-02-25 06:17:50,83.0,1
3,65879,2610228,2015-05-26 08:11:45,Abnormal Update Rate Aftertreatment 1 Intake NOx,04993120*00001782*082113134117*07700053*I0*BBZ*,79419774,6X1u10D1500000000,CMMNS,0,3216,...,-86.433842,2015-05-21 14:40:18,NaT,2015-06-08 19:37:07,13.0,0,NaT,2015-02-25 06:17:50,90.0,1
4,65939,2611189,2015-05-26 08:44:10,Abnormal Update Rate Aftertreatment 1 Intake NOx,04993120*00001782*082113134117*07700053*I0*BBZ*,79419774,6X1u10D1500000000,CMMNS,0,3216,...,-86.440787,2015-05-26 08:52:22,NaT,2015-06-08 19:37:07,13.0,0,NaT,2015-02-25 06:17:50,90.0,1


In [8]:
#Remove columns we don't think we need
faults = faults.drop(['ESS_Id','ecuSoftwareVersion','ecuSerialNumber','ecuModel','ecuMake','ecuSource','active','MCTNumber','Latitude','Longitude','LocationTimeStamp'], axis = 1)
#faults = faults.drop(['ecuModel'], axis = 1)

In [9]:
#Read in the corresponding diagnostics data
diagnostics = pd.read_csv('../data/VehicleDiagnosticOnboardData.csv')

In [10]:
#Flatten the diagnostics
diag_pivot = diagnostics.pivot_table('Value','FaultId','Name',aggfunc = 'first')

In [11]:
#Merge to the fault data
faults = faults.merge(diag_pivot, how = 'left', left_on = 'RecordID', right_on = 'FaultId')

In [12]:
#How many unique spn values
print(faults.shape)
print(faults['spn'].nunique())

(205610, 39)
170


In [13]:
#Extract the spns so we can flatten those
fault_mode = 2 #1 = spn+fmi / 2 = spn only
faults_pivot = faults[faults['spn'] != 5246][['RecordID','spn','fmi']]
#Make a key of the spn + fmi if fault_mode = 1, otherwise just use spn
if fault_mode == 1:
    faults_pivot['SpnFmi'] = faults_pivot['spn'].astype('str')+'-'+faults_pivot['fmi'].astype('str')
elif fault_mode == 2:
    faults_pivot['SpnFmi'] = faults_pivot['spn'].astype('str')
#Drop the original columns
faults_pivot = faults_pivot[['RecordID','SpnFmi']]
print(faults_pivot.shape)
print(faults_pivot['SpnFmi'].nunique())

(205330, 2)
169


In [14]:
#lots of values with low number of observations
faults_pivot['SpnFmi'].value_counts()

111     157668
97       11294
1569      4328
1761      2981
641       1995
         ...  
5569         1
677          1
5321         1
3695         1
1327         1
Name: SpnFmi, Length: 169, dtype: int64

In [15]:
#Drop all with overall frequency less than 100
#faults_pivot = faults_pivot.groupby('SpnFmi').filter(lambda x : len(x) > 99)
#Drop all with overall frequency less than parameter
faults_pivot = faults_pivot.groupby('SpnFmi').filter(lambda x : len(x) > 2)
#Get dummies
faults_pivot = pd.get_dummies(faults_pivot)
#This will drop all the 5246 rows plus any the rows for faults we excluded above due to low counts
faults = faults.merge(faults_pivot, how = 'inner')
faults.head()
#Note that the dummies merge as floats instead of ints if we left join above because we are introducing NaNs in that case
#We are inner joining now because we have all our 5246 faults encoded as a NextDerateTimestamp
#InFailureWindow is our eventual target variable (at this point!)

,RecordID,EventTimeStamp,eventDescription,spn,fmi,activeTransitionCount,EquipmentID,NextDerateTimestamp,MaxTimestamp,DateDiff,...,SpnFmi_723,SpnFmi_7827,SpnFmi_7854,SpnFmi_81,SpnFmi_84,SpnFmi_862,SpnFmi_91,SpnFmi_95,SpnFmi_96,SpnFmi_97
0,4968,2015-02-25 06:17:50,Low (Severity Low) Engine Coolant Level,111,17,1,1327,NaT,2015-06-08 19:37:07,103.0,...,0,0,0,0,0,0,0,0,0,0
1,4969,2015-02-25 06:17:50,Low (Severity Medium) Battery Potential / Powe...,444,18,1,1327,NaT,2015-06-08 19:37:07,103.0,...,0,0,0,0,0,0,0,0,0,0
2,58809,2015-05-19 12:02:55,Abnormal Update Rate Aftertreatment 1 Intake NOx,3216,9,1,1327,NaT,2015-06-08 19:37:07,20.0,...,0,0,0,0,0,0,0,0,0,0
3,65879,2015-05-26 08:11:45,Abnormal Update Rate Aftertreatment 1 Intake NOx,3216,9,1,1327,NaT,2015-06-08 19:37:07,13.0,...,0,0,0,0,0,0,0,0,0,0
4,65939,2015-05-26 08:44:10,Abnormal Update Rate Aftertreatment 1 Intake NOx,3216,9,1,1327,NaT,2015-06-08 19:37:07,13.0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
col_sel = [col for col in faults.columns if 'SpnFmi_' in col]
#print(list(faults.columns))
print(col_sel)

['SpnFmi_100', 'SpnFmi_101', 'SpnFmi_102', 'SpnFmi_103', 'SpnFmi_105', 'SpnFmi_1072', 'SpnFmi_1075', 'SpnFmi_108', 'SpnFmi_110', 'SpnFmi_111', 'SpnFmi_1127', 'SpnFmi_1172', 'SpnFmi_1176', 'SpnFmi_1209', 'SpnFmi_1239', 'SpnFmi_125', 'SpnFmi_1322', 'SpnFmi_1323', 'SpnFmi_1325', 'SpnFmi_1326', 'SpnFmi_1328', 'SpnFmi_1347', 'SpnFmi_1569', 'SpnFmi_157', 'SpnFmi_168', 'SpnFmi_171', 'SpnFmi_175', 'SpnFmi_1761', 'SpnFmi_188', 'SpnFmi_190', 'SpnFmi_191', 'SpnFmi_196608', 'SpnFmi_237', 'SpnFmi_2623', 'SpnFmi_27', 'SpnFmi_2791', 'SpnFmi_3031', 'SpnFmi_3058', 'SpnFmi_3060', 'SpnFmi_3216', 'SpnFmi_3218', 'SpnFmi_3222', 'SpnFmi_3226', 'SpnFmi_3228', 'SpnFmi_3242', 'SpnFmi_3246', 'SpnFmi_3251', 'SpnFmi_3360', 'SpnFmi_3361', 'SpnFmi_3362', 'SpnFmi_3363', 'SpnFmi_3364', 'SpnFmi_3464', 'SpnFmi_3480', 'SpnFmi_3482', 'SpnFmi_3490', 'SpnFmi_3509', 'SpnFmi_3510', 'SpnFmi_3511', 'SpnFmi_3512', 'SpnFmi_3513', 'SpnFmi_3515', 'SpnFmi_3556', 'SpnFmi_3597', 'SpnFmi_3610', 'SpnFmi_3703', 'SpnFmi_3720', 'SpnFmi_393

In [17]:
summary_window_str = str(summary_window)+'d'
#This did not work if I just set the index to the timestamp and grouped by the EquipmentID. No idea why.
#Also, even though documentation says you can use on='EventTimeStamp' in the rolling function, that will not work either.
faults = faults.set_index(['EquipmentID','EventTimeStamp'])
for col in col_sel:
    newcol = col.replace('SpnFmi', 'Summ')
    faults[newcol]= np.where(faults['InSummWindow']==1, 
                              faults.reset_index(level='EquipmentID').groupby(['EquipmentID'])[col].rolling(summary_window_str).sum(),
                                0)
#Drop our original dummy columns, which have now been summarized
faults = faults.drop(columns = col_sel)
#Drop the rows not in the summary window. They also have been summarized if appropriate, but they are not valid
#as observations because they have less than the correct time period needed to aggregate a summary on them
faults = faults[faults['InSummWindow']==1]
faults.head()

RecordID  \
EquipmentID EventTimeStamp                  
1327        2015-05-19 12:02:55     58809   
            2015-05-26 08:11:45     65879   
            2015-05-26 08:44:10     65939   
            2015-05-28 09:45:18     68619   
            2015-06-03 15:24:27     75366   

                                                                 eventDescription  \
EquipmentID EventTimeStamp                                                          
1327        2015-05-19 12:02:55  Abnormal Update Rate Aftertreatment 1 Intake NOx   
            2015-05-26 08:11:45  Abnormal Update Rate Aftertreatment 1 Intake NOx   
            2015-05-26 08:44:10  Abnormal Update Rate Aftertreatment 1 Intake NOx   
            2015-05-28 09:45:18  Condition Exists Engine Protection Torque Derate   
            2015-06-03 15:24:27  Condition Exists Engine Protection Torque Derate   

                                  spn  fmi  activeTransitionCount  \
EquipmentID EventTimeStamp                                          
1327        2015-05-19 12:02:55  3216    9                      1   
            2015-05-26 08:11:45  3216    9                      1   
            2015-05-26 08:44:10  3216    9                      1   
            2015-05-28 09:45:18  1569   31                      1   
            2015-06-03 15:24:27  1569   31                      1   

                                NextDerateTimestamp        MaxTimestamp  \
EquipmentID EventTimeStamp                                                
1327        2015-05-19 12:02:55                 NaT 2015-06-08 19:37:07   
            2015-05-26 08:11:45                 NaT 2015-06-08 19:37:07   
            2015-05-26 08:44:10                 NaT 2015-06-08 19:37:07   
            2015-05-28 09:45:18                 NaT 2015-06-08 19:37:07   
            2015-06-03 15:24:27                 NaT 2015-06-08 19:37:07   

                                 DateDiff  InFailureWindow  \
EquipmentID EventTimeStamp                                   
1327        2015-05-19 12:02:55      20.0                0   
            2015-05-26 08:11:45      13.0                0   
            2015-05-26 08:44:10      13.0                0   
            2015-05-28 09:45:18      11.0                0   
            2015-06-03 15:24:27       5.0                0   

                                PrevDerateTimestamp  ... Summ_723  Summ_7827  \
EquipmentID EventTimeStamp                           ...                       
1327        2015-05-19 12:02:55                 NaT  ...      0.0        0.0   
            2015-05-26 08:11:45                 NaT  ...      0.0        0.0   
            2015-05-26 08:44:10                 NaT  ...      0.0        0.0   
            2015-05-28 09:45:18                 NaT  ...      0.0        0.0   
            2015-06-03 15:24:27                 NaT  ...      0.0        0.0   

                                 Summ_7854 Summ_81 Summ_84 Summ_862 Summ_91  \
EquipmentID EventTimeStamp                                                    
1327        2015-05-19 12:02:55        0.0     0.0     0.0      0.0     0.0   
            2015-05-26 08:11:45        0.0     0.0     0.0      0.0     0.0   
            2015-05-26 08:44:10        0.0     0.0     0.0      0.0     0.0   
            2015-05-28 09:45:18        0.0     0.0     0.0      0.0     0.0   
            2015-06-03 15:24:27        0.0     0.0     0.0      0.0     0.0   

                                Summ_95 Summ_96 Summ_97  
EquipmentID EventTimeStamp                               
1327        2015-05-19 12:02:55     0.0     0.0     0.0  
            2015-05-26 08:11:45     0.0     0.0     0.0  
            2015-05-26 08:44:10     0.0     0.0     0.0  
            2015-05-28 09:45:18     0.0     0.0     0.0  
            2015-06-03 15:24:27     0.0     0.0     0.0  

[5 rows x 178 columns]

In [18]:
faults.shape

(201857, 178)

The below is the only example I found on stack overflow of how to make the rolling function work for irregular time intervals
https://stackoverflow.com/questions/48956286/pandas-multiindex-dataframe-rolling-offset
My preference would be to use the `on` argument for rolling but I could not get it to work.

`rng = pd.date_range('2017-01-03', periods=20, freq='8D')
i = pd.MultiIndex.from_product([['A','B','C'], rng], names=['Name','Date'])
df = pd.DataFrame(np.random.randn(60), i, columns=['Vals'])
df['Avg'] = df.groupby(['Name'])['Vals'].rolling('30D').mean() `

`rng = pd.date_range('2017-01-03', periods=20, freq='8D')
i = pd.MultiIndex.from_product([['A','B','C'], rng], names=['Name','Date'])
df = pd.DataFrame(np.random.randn(60), i, columns=['Vals'])
df['Avg'] = df.reset_index(level='Name').groupby(['Name'])['Vals'].rolling('30D').mean()`

In [19]:
faults.loc['306'].head()

,RecordID,eventDescription,spn,fmi,activeTransitionCount,NextDerateTimestamp,MaxTimestamp,DateDiff,InFailureWindow,PrevDerateTimestamp,...,Summ_723,Summ_7827,Summ_7854,Summ_81,Summ_84,Summ_862,Summ_91,Summ_95,Summ_96,Summ_97
EventTimeStamp,,,,,,,,,,,,,,,,,,,,,
2016-01-11 05:05:09,340897,Low (Severity Low) Engine Coolant Level,111,17,2,2018-09-07 11:22:40,2019-03-05 00:34:49,970.0,0,NaT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-05-14 00:42:07,475695,High (Severity Low) Aftertreatment Diesel Part...,3936,15,1,2018-09-07 11:22:40,2019-03-05 00:34:49,846.0,0,NaT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-05-18 05:20:17,478875,Low (Severity Low) Engine Coolant Level,111,17,2,2018-09-07 11:22:40,2019-03-05 00:34:49,842.0,0,NaT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-05-19 05:17:53,479898,Low (Severity Low) Engine Coolant Level,111,17,3,2018-09-07 11:22:40,2019-03-05 00:34:49,841.0,0,NaT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-05-20 05:17:00,480940,Low (Severity Low) Engine Coolant Level,111,17,4,2018-09-07 11:22:40,2019-03-05 00:34:49,840.0,0,NaT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
faults['InFailureWindow'].sum()

1487

In [21]:
faults.shape

(201857, 178)

In [22]:
3748/192832

0.01943660803186193

In [23]:
obj_to_flt = ['BarometricPressure', 'DistanceLtd', 'EngineCoolantTemperature', 
                       'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 
                       'FuelLtd', 'FuelRate', 'IntakeManifoldTemperature', 
                       'SwitchedBatteryVoltage', 'TurboBoostPressure']

In [24]:
faults[obj_to_flt] = faults[obj_to_flt].apply(pd.to_numeric, errors='coerce')

In [25]:
faults[obj_to_flt].info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 201857 entries, ('1327', Timestamp('2015-05-19 12:02:55')) to ('310', Timestamp('2018-06-05 20:09:14'))
Data columns (total 13 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   BarometricPressure         199161 non-null  float64
 1   DistanceLtd                199162 non-null  float64
 2   EngineCoolantTemperature   199087 non-null  float64
 3   EngineLoad                 198882 non-null  float64
 4   EngineOilPressure          199184 non-null  float64
 5   EngineOilTemperature       198466 non-null  float64
 6   EngineRpm                  199287 non-null  float64
 7   EngineTimeLtd              198671 non-null  float64
 8   FuelLtd                    198978 non-null  float64
 9   FuelRate                   198808 non-null  float64
 10  IntakeManifoldTemperature  199206 non-null  float64
 11  SwitchedBatteryVoltage     35245 non-null   float64
 12  TurboBoost

In [32]:
use_cols = ['InFailureWindow','NextDerateTimestamp']
[use_cols.append(flt) for flt in obj_to_flt]
[use_cols.append(col) for col in faults.columns if 'Summ_' in col]
len(use_cols)

156

In [33]:
faults_15_15 = faults[use_cols]
faults_15_15.head()

InFailureWindow NextDerateTimestamp  \
EquipmentID EventTimeStamp                                             
1327        2015-05-19 12:02:55                0                 NaT   
            2015-05-26 08:11:45                0                 NaT   
            2015-05-26 08:44:10                0                 NaT   
            2015-05-28 09:45:18                0                 NaT   
            2015-06-03 15:24:27                0                 NaT   

                                 BarometricPressure  DistanceLtd  \
EquipmentID EventTimeStamp                                         
1327        2015-05-19 12:02:55              14.355     520309.6   
            2015-05-26 08:11:45                 NaN          NaN   
            2015-05-26 08:44:10              14.355     520314.6   
            2015-05-28 09:45:18                 NaN          NaN   
            2015-06-03 15:24:27                 NaN          NaN   

                                 EngineCoolantTemperature  EngineLoad  \
EquipmentID EventTimeStamp                                              
1327        2015-05-19 12:02:55                     185.0         0.0   
            2015-05-26 08:11:45                       NaN         NaN   
            2015-05-26 08:44:10                     161.6        45.0   
            2015-05-28 09:45:18                       NaN         NaN   
            2015-06-03 15:24:27                       NaN         NaN   

                                 EngineOilPressure  EngineOilTemperature  \
EquipmentID EventTimeStamp                                                 
1327        2015-05-19 12:02:55              31.90              203.8438   
            2015-05-26 08:11:45                NaN                   NaN   
            2015-05-26 08:44:10              38.86              154.5125   
            2015-05-28 09:45:18                NaN                   NaN   
            2015-06-03 15:24:27                NaN                   NaN   

                                 EngineRpm  EngineTimeLtd  ...  Summ_723  \
EquipmentID EventTimeStamp                                 ...             
1327        2015-05-19 12:02:55   1083.125       10374.80  ...       0.0   
            2015-05-26 08:11:45        NaN            NaN  ...       0.0   
            2015-05-26 08:44:10   1170.000       10375.15  ...       0.0   
            2015-05-28 09:45:18        NaN            NaN  ...       0.0   
            2015-06-03 15:24:27        NaN            NaN  ...       0.0   

                                 Summ_7827  Summ_7854  Summ_81  Summ_84  \
EquipmentID EventTimeStamp                                                
1327        2015-05-19 12:02:55        0.0        0.0      0.0      0.0   
            2015-05-26 08:11:45        0.0        0.0      0.0      0.0   
            2015-05-26 08:44:10        0.0        0.0      0.0      0.0   
            2015-05-28 09:45:18        0.0        0.0      0.0      0.0   
            2015-06-03 15:24:27        0.0        0.0      0.0      0.0   

                                 Summ_862  Summ_91  Summ_95  Summ_96  Summ_97  
EquipmentID EventTimeStamp                                                     
1327        2015-05-19 12:02:55       0.0      0.0      0.0      0.0      0.0  
            2015-05-26 08:11:45       0.0      0.0      0.0      0.0      0.0  
            2015-05-26 08:44:10       0.0      0.0      0.0      0.0      0.0  
            2015-05-28 09:45:18       0.0      0.0      0.0      0.0      0.0  
            2015-06-03 15:24:27       0.0      0.0      0.0      0.0      0.0  

[5 rows x 156 columns]

In [34]:
faults_15_15.shape

(201857, 156)

In [35]:
#faults_15_15.reset_index()

In [36]:
#faults_15_15.head()

In [37]:
#faults_15_15.to_csv('../data/faults_15_15.csv')
faults_15_15.to_csv('../data/faults_2_2.csv')

Potentially critical codes
5394
5392
1761
4094
4334
4339
